# COMP 562 Final Project: Facial Landmark Detection

The objective of the project is to build a CNN model to detect facial landmark given a human face image. 

In [ ]:
import pickle
import csv
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.layers import Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, History
from keras.optimizers import Adam

## Load and augumenting data
The pickle file stores the images and the corresponding facial landmarks that was preprocessed using the code in the preprocessing folder. The size of X_train is 3148*160*160*3, which means there are 3148 RGB images whose size are 160*160. The size of Y_train is 3148*136, which stores the 68 keypoints of those 3148 images.

In [ ]:
unpickled_df = pd.read_pickle("/300W_train_160by160.pickle")

X_train0 = unpickled_df['X']
Y_train0 = unpickled_df['Y']['kpt_norm']

## Data Augmentaion
Howerver, for a project like this, 3148 is not large enough to train a strong model. So we need to apply some tricks to create more data. Here we use noising and bluring. 

In [ ]:
#augmentation: noisy
temp1 = X_train0.copy()
for t in range(3148):
    noise = np.random.randint(20, size = (160,160, 3), dtype = 'uint8')
    temp = temp1[t]
    for i in range(160):
        for j in range(160):
            for k in range(3):
                if (temp[i,j,k] < 235):
                    temp[i,j,k] += noise[i,j,k]

## First Attempt: structures similar with VGG/AlexNet 
This is the first attempt, I designed models that similar with VGG and ALexNet, but both are not working.

In [ ]:
model = Sequential()
model.add(Conv2D(filters=96, kernel_size=11, activation='relu', input_shape=(160, 160,3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

model.add(Conv2D(filters=256, kernel_size=11, strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

model.add(Conv2D(filters=384, kernel_size=11, activation='relu'))

model.add(Conv2D(filters=384, kernel_size=11, activation='relu'))

model.add(Conv2D(filters=256, kernel_size=11, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

model.add(Flatten())

model.add(Dense(4096, activation='relu',kernel_initializer='glorot_normal'))
model.add(Dropout(0.4))

model.add(Dense(4096, activation='relu',kernel_initializer='glorot_normal'))
model.add(Dropout(0.4))

model.add(Dense(1000, activation='relu',kernel_initializer='glorot_normal'))
model.add(Dropout(0.4))

model.add(Dense(136, activation='softmax'))


model.summary()



hist = History()
epochs = 150
batch_size = 64

#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

checkpointer = ModelCheckpoint(filepath='weights.final_2.hdf5', 
                               verbose=1, save_best_only=True)

## TODO: Compile the model
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='mean_absolute_percentage_error', metrics=['accuracy'])

hist_final = model.fit(X_train0, Y_train0, validation_split=0.2,
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, hist,tbCallBack], verbose=1)

## Attempt2: A shallow model that detects 68 points at a same time
This is a shallower CNN

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=3, activation='relu', input_shape=(160, 160,3)))

model.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(256, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(256, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(136))


# Summarize the model
model.summary()

hist = History()
epochs = 150
batch_size = 64

#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

checkpointer = ModelCheckpoint(filepath='weights.final_2.hdf5', 
                               verbose=1, save_best_only=True)

## TODO: Compile the model
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='mean_absolute_percentage_error', metrics=['accuracy'])

hist_final = model.fit(X_train0, Y_train0, validation_split=0.2,
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, hist,tbCallBack], verbose=1)


model.save('att2.h5')

## Attempt3: Divide and conquer:
Using 4 models, each only detect some small number of points

In [ ]:
#model one, for the first 17 points
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=3, activation='relu', input_shape=(160, 160,3)))

model.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(256, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(256, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(256, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(34))


# Summarize the model
model.summary()

hist = History()
epochs = 80
batch_size = 64

#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

checkpointer = ModelCheckpoint(filepath='weights.final_2.hdf5', 
                               verbose=1, save_best_only=True)

## TODO: Compile the model
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='mean_absolute_percentage_error', metrics=['accuracy'])

hist_final = model.fit(X_train0, Y_train1, validation_split=0.2,
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, hist,tbCallBack], verbose=1)


model.save('m1.h5')

In [ ]:
#model 2, for 10 points on the eyebrow
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=3, activation='relu', input_shape=(160, 160,3)))

model.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(256, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(256, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(256, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(20))


# Summarize the model
model.summary()

hist = History()
epochs = 150
batch_size = 64

#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

checkpointer = ModelCheckpoint(filepath='weights.final_2.hdf5', 
                               verbose=1, save_best_only=True)

## TODO: Compile the model
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='mean_absolute_percentage_error', metrics=['accuracy'])

hist_final = model.fit(X_train0, Y_train2, validation_split=0.2,
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, hist,tbCallBack], verbose=1)


model.save('m2.h5')

In [ ]:
#model 3, for eyes and nose
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=3, activation='relu', input_shape=(160, 160,3)))

model.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(256, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(256, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(256, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(42))


# Summarize the model
model.summary()

hist = History()
epochs = 150
batch_size = 64

#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

checkpointer = ModelCheckpoint(filepath='weights.final_2.hdf5', 
                               verbose=1, save_best_only=True)

## TODO: Compile the model
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='mean_absolute_percentage_error', metrics=['accuracy'])

hist_final = model.fit(X_train0, Y_train3, validation_split=0.2,
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, hist,tbCallBack], verbose=1)


model.save('m3.h5')

In [ ]:
# fourth model, for mouth
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=3, activation='relu', input_shape=(160, 160,3)))

model.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(256, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(256, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(256, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(512, activation='relu',kernel_initializer='glorot_normal'))

model.add(Dense(40))


# Summarize the model
model.summary()

hist = History()
epochs = 150
batch_size = 64

#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

checkpointer = ModelCheckpoint(filepath='weights.final_2.hdf5', 
                               verbose=1, save_best_only=True)

## TODO: Compile the model
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='mean_absolute_percentage_error', metrics=['accuracy'])

hist_final = model.fit(X_train0, Y_train4, validation_split=0.2,
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, hist,tbCallBack], verbose=1)


model.save('m4.h5')

### predict and plot 

In [ ]:
img=plt.imread("/content/ro160.jpg")
im = np.zeros((1,160,160,3))
im[0,:,:,:] = img[:]
points = model.predict(im)
plt.imshow(img)
for i in range(int(len(points[0])/2)):
    plt.plot(points[0][2*i]*160,points[0][2*i+1]*160,'ro')
from keras.models import load_model
img=plt.imread("/content/ro160.jpg")
im = np.zeros((1,160,160,3))
im[0,:,:,:] = img[:]
model1 = load_model('my_model_final.h5')
model2 = load_model('my_model_final2.h5')
model3 = load_model('my_model_final3.h5')
model4 = load_model('my_model_final4.h5')
point1 = model1.predict(im)
point2 = model2.predict(im)
point3 = model3.predict(im)
point4 = model4.predict(im)
plt.imshow(img)
for i in range(int(len(point1[0])/2)):
    plt.plot(point1[0][2*i]*160,point1[0][2*i+1]*160,'ro')
for i in range(int(len(point2[0])/2)):
    plt.plot(point2[0][2*i]*160,point2[0][2*i+1]*160,'ro')
for i in range(int(len(point3[0])/2)):
    plt.plot(point3[0][2*i]*160,point3[0][2*i+1]*160,'ro')
for i in range(int(len(point4[0])/2)):
    plt.plot(point4[0][2*i]*160,point4[0][2*i+1]*160,'ro')